<a href="https://colab.research.google.com/github/Tessellate-Imaging/monk_v1/blob/master/study_roadmaps/4_image_classification_zoo/Classifier%20-%20State%20Farm%20Distracted%20Driver%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## Install Monk


## Using pretrained model for State Farm Distracted Driver Detection dataset


## Training a classifier from scratch

# Install Monk

  - git clone https://github.com/Tessellate-Imaging/monk_v1.git

  - cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt
        (Select the requirements file as per OS and CUDA version)


In [ ]:
! git clone https://github.com/Tessellate-Imaging/monk_v1.git

In [ ]:
# If using Colab install using the commands below
! cd monk_v1/installation/Misc && pip install -r requirements_colab.txt

# If using Kaggle uncomment the following command
#! cd monk_v1/installation/Misc && pip install -r requirements_kaggle.txt

# Select the requirements file as per OS and CUDA version when using a local system or cloud
#! cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt



# Used trained classifier for demo

In [ ]:
# Import monk

import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Download trained weights

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1EGhQawPH-NN4S3TQpcuZHPgcNGuqmBN6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1EGhQawPH-NN4S3TQpcuZHPgcNGuqmBN6" -O cls_distraction_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq cls_distraction_trained.zip 

In [ ]:
ls workspace/Project-Driver

In [ ]:
# Keras project
from keras_prototype import prototype

In [ ]:
# Load project in inference mode

gtf = prototype(verbose=1);
gtf.Prototype("Project-Driver", "Keras-resnet50_v2", eval_infer=True);

#Other trained models - uncomment 
#gtf.Prototype("Project-Driver", "Keras-resnet101_v2", eval_infer=True);

In [ ]:
label_map = {
    "c0": "safe driving",
    "c1": "texting - right",
    "c2": "talking on the phone - right",
    "c3": "texting - left",
    "c4": "talking on the phone - left",
    "c5": "operating the radio",
    "c6": "drinking",
    "c7": "reaching behind",
    "c8": "hair and makeup",
    "c9": "talking to passenger"
}

In [ ]:
# Infer

In [ ]:
img_name = "workspace/test/1.jpg"
predictions = gtf.Infer(img_name=img_name);
print("label mapped to - ", label_map[predictions["predicted_class"]])
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/3.jpg"
predictions = gtf.Infer(img_name=img_name);
print("label mapped to - ", label_map[predictions["predicted_class"]])
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/4.jpg"
predictions = gtf.Infer(img_name=img_name);
print("label mapped to - ", label_map[predictions["predicted_class"]])
from IPython.display import Image
Image(filename=img_name)

# Training custom classifier from scratch

## Dataset
    - Credits: https://www.kaggle.com/c/state-farm-distracted-driver-detection/data

## Download
    - sign up for the competition to download the data

In [ ]:
! pip install kaggle

In [ ]:
! kaggle competitions download -c state-farm-distracted-driver-detection

In [ ]:
! unzip -qq state-farm-distracted-driver-detection.zip -d driver_dataset

## Training

In [ ]:
# Monk
import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Using mxnet-gluon backend 
#from gluon_prototype import prototype

# For pytorch backend
#from pytorch_prototype import prototype

# For Keras backend
from keras_prototype import prototype

In [ ]:
# Create Project and Experiment

gtf = prototype(verbose=1);
gtf.Prototype("Project-Driver", "Keras-resnet50_v2");

In [ ]:
gtf.Default(dataset_path="driver_dataset/imgs/train/",
            model_name="resnet50_v2", 
            freeze_base_network=False,
            num_epochs=2);

### How to change hyper parameters and models 
  - Docs - https://github.com/Tessellate-Imaging/monk_v1#4
  - Examples - https://github.com/Tessellate-Imaging/monk_v1/tree/master/study_roadmaps/1_getting_started_roadmap

In [ ]:
#Start Training
gtf.Train();

#Read the training summary generated once you run the cell and training is completed

## Testing on a new dataset for submission

In [ ]:
# Import monk

import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Using mxnet-gluon backend 
#from gluon_prototype import prototype

# For pytorch backend
#from pytorch_prototype import prototype

# For Keras backend
from keras_prototype import prototype

In [ ]:
# Create Project and Experiment

gtf = prototype(verbose=1);
gtf.Prototype("Project-Driver", "Keras-resnet50_v2", eval_infer=True);

In [ ]:
ls driver_dataset

In [ ]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from scipy.special import softmax
df = pd.read_csv("driver_dataset/sample_submission.csv")

In [ ]:
for i in tqdm(range(len(df))):
    img_name = "driver_dataset/imgs/test/" + df["img"][i];
    
    #Invoking Monk's nferencing engine inside a loop
    predictions = gtf.Infer(img_name=img_name, return_raw=True);
    out = predictions["raw"]
    
    df["c0"][i] = out[0];
    df["c1"][i] = out[1];
    df["c2"][i] = out[2];
    df["c3"][i] = out[3];
    df["c4"][i] = out[4];
    df["c5"][i] = out[5];
    df["c6"][i] = out[6];
    df["c7"][i] = out[7];
    df["c8"][i] = out[8];
    df["c9"][i] = out[9];

In [ ]:
df.to_csv("submission.csv", index=False);